In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import os
from ase.io import read

from ml_utils.util import flatten, remove_empty_from_array

In [ ]:
path2target = '/Users/y1u0d2/desktop/Lab/data/qe_data/Si/mp-168/0oA5IvYoOQRatauE3q'

In [ ]:
# get energy from lmp output
lmp_out_filename = 'log.lammps'
with open(os.path.join(path2target, lmp_out_filename), mode='r') as f:
    lines = [s.strip() for s in f.readlines()]


info_header_idx = None
for i, line in enumerate(lines):
    if 'TotEng' in line:
        info_header_idx = i
        break

info_from_lmp_log = [remove_empty_from_array(line.split(' ')) for line in lines[info_header_idx: info_header_idx+2]]
lmp_log_df = pd.DataFrame(columns=info_from_lmp_log[0], data=info_from_lmp_log[1:], dtype=float)
total_energy = lmp_log_df['TotEng'].values[0]


In [ ]:
from ml_utils.util import get_param_idx


with open(os.path.join(path2target, 'dump.out'), mode='r') as f:
    lines = [s.strip() for s in f.readlines()]
    
num_atom_idx = get_param_idx('NUMBER OF ATOMS', lines)
atom_idx = get_param_idx('ITEM: ATOMS', lines)

num_atom = int(lines[num_atom_idx+1])
atom_header = lines[atom_idx].split(' ')[2:]
atom_coords = [i.split(' ') for i in lines[atom_idx+1: atom_idx+1+num_atom]]
atom_coords = [list(map(float, i)) for i in atom_coords]
atom_df = pd.DataFrame(data=atom_coords, columns=atom_header)

In [ ]:
atom_df[['fx', 'fy', 'fz']].values

In [ ]:
from scf.get_lattice_info import QELattice

qel = QELattice(path2target)

In [ ]:
qel.get_force()

In [ ]:
atom_df[['fx', 'fy', 'fz']].values

In [ ]:
qel.get_force() - atom_df[['fx', 'fy', 'fz']].values

In [ ]:
from create_input.lmp.info_from_lmp_out import LammpsLogParser

In [ ]:
lmp_log_parser = LammpsLogParser(
    path2target=path2target,
    dump_filename='dump.out'
)

In [ ]:
lmp_log_parser.get_energy()

In [ ]:
from create_input.from_scf_minus_lmp import ScfMinusLammpsN2p2
from create_input.lmp.info_from_lmp_out import LammpsLogParser

lmp_log_parser = LammpsLogParser(
    path2target=path2target,
    dump_filename='dump.out'
)
obj = ScfMinusLammpsN2p2(path2scfout=path2target, dump_filename='dump.out', is_comment=True)
list(obj.n2p2_block)

In [19]:
# エネルギーの差
qel.get_energy() - lmp_log_parser.get_energy()

-5032.983302024

In [20]:
# 力の差
qel.get_force() - atom_df[['fx', 'fy', 'fz']].values

array([[ -1.96810539,   1.3488064 ,  -0.73029153],
       [-14.09436279,  -0.97555655,  -2.78097409],
       [ 12.36779033, -14.06862881, -10.81049956],
       [  0.91343099,  -1.08891656, -11.01745442],
       [-20.03396353,   2.4344292 ,   9.35471744],
       [ 14.49297147,  -1.85054195,   3.09692067],
       [ 12.15347213,  15.16967487,   0.67098572],
       [ -3.83123321,  -0.96920662,  12.21658552]])